# CISSM Pipeline.

## Environment Setup.
Estas librerías proporcionan la base para realizar el procesamiento y análisis de datos de forma eficiente y visualmente comprensible. Su uso combinado permite realizar tareas desde la manipulación de datos hasta su visualización de manera fluida y efectiva en el entorno de Jupyter Notebook.
1. **Pandas**: Librería de Python especializada en el manejo y análisis de datos tabulares. Permite realizar operaciones de transformación, agregación, limpieza y análisis de datos de manera eficiente.

2. **Numpy**: Librería fundamental para la computación científica en Python. Ofrece soporte para arrays multidimensionales, matrices y funciones matemáticas avanzadas.

3. **IPython.display**: Funciones útiles para mostrar contenido interactivo y enriquecido en los notebooks. Con estas funciones, se puede renderizar HTML, mostrar tablas y gráficos de manera directa en el cuaderno de Jupyter.

In [12]:
import pandas as pd
import requests
from time import sleep
import random
import json

## Pipeline Process: Extract, Transform y Load (ETL).
El pipeline de procesamiento de datos se estructura en tres pasos principales: **Extract**, **Transform** y **Load**. Este flujo se automatiza y gestiona utilizando **Azure Data Factory** (ADF) y se complementa con **Azure Databricks** para realizar el procesamiento y transformación de datos de manera eficiente.

1. La primera fase del pipeline consiste en la **extracción** de datos desde diversas fuentes. Estas fuentes pueden incluir bases de datos, archivos en la nube o sistemas externos.

2. Una vez que los datos han sido extraídos, el siguiente paso es la **transformación**. Este paso incluye la limpieza, agregación y modificación de los datos para que sean adecuados para el análisis.

3. Finalmente, después de que los datos han sido transformados, llega el paso de la **carga**. Este paso implica mover los datos procesados a su destino final, que puede ser una base de datos, un sistema de almacenamiento o un sistema de análisis.

### **01. Extract.**
- **Azure Data Factory** gestiona la conexión a las fuentes de datos.
- Se pueden programar procesos de extracción de manera recurrente, asegurando que siempre se tenga acceso a los datos más recientes.
- La extracción de datos se puede automatizar con **triggers** o eventos programados para iniciar el flujo de datos.

In [4]:
CURRENT_DATE = '2024-04-01'

#### Generate row intervals.
Este código divide un rango de filas (hasta 600) en intervalos de 100 filas. Crea un DataFrame con dos columnas, `start_row` y `end_row`, que representan los límites de cada intervalo. Esto permite procesar los datos en bloques más pequeños. Finalmente, convierte los valores de las columnas a cadenas de texto y muestra los primeros intervalos.

In [7]:
max_rows = 600  # Max extraction limit
step = 100
start_rows = range(0, max_rows, step)
end_rows = [min(start_row + step, max_rows) for start_row in start_rows]

df_row_intervals = pd.DataFrame({'start_row': start_rows, 'end_row': end_rows})
df_row_intervals = df_row_intervals.astype(str)
df_row_intervals.head()

,start_row,end_row
0,0,100
1,100,200
2,200,300
3,300,400
4,400,500


#### Generate links.
Este código genera una lista de URLs, reemplazando los valores de `start_row` y `end_row` en una plantilla de URL para cada intervalo de filas en el DataFrame. Cada URL corresponde a un intervalo de filas específico, y la lista resultante se guarda en la variable `links`.

In [11]:
url_patt = "https://cissm.liquifiedapps.com/api/?summary=1&start_row={start_row}&end_row={end_row}&filter={{}}&sort=[]"
links = df_row_intervals.apply(lambda x: url_patt.format(start_row=x['start_row'], end_row=x['end_row']), axis=1).to_list()
links

['https://cissm.liquifiedapps.com/api/?summary=1&start_row=0&end_row=100&filter={}&sort=[]',
 'https://cissm.liquifiedapps.com/api/?summary=1&start_row=100&end_row=200&filter={}&sort=[]',
 'https://cissm.liquifiedapps.com/api/?summary=1&start_row=200&end_row=300&filter={}&sort=[]',
 'https://cissm.liquifiedapps.com/api/?summary=1&start_row=300&end_row=400&filter={}&sort=[]',
 'https://cissm.liquifiedapps.com/api/?summary=1&start_row=400&end_row=500&filter={}&sort=[]',
 'https://cissm.liquifiedapps.com/api/?summary=1&start_row=500&end_row=600&filter={}&sort=[]']

#### Make requests.

In [13]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.41',
    'Content-Type': 'text/html',
}

In [ ]:
incidents_table = []

for i, url in enumerate(links):
    print(url)
    sleep(random.randint(1, 60))
    page = requests.get(url, headers=headers)
    if page.status_code != 200:
        print(f"Failed to retrieve data from {url}. HTTP Status Code: {page.status_code}")
        continue  # Salta esta iteración y continúa con la siguiente URL
    page_content = page.content.decode("utf-8")
    if not page_content:
        print(f"Empty response for URL: {url}")
        continue
    try:
        incidents = json.loads(page_content)['rows']
        if incidents == False:
            incidents = []

    except json.JSONDecodeError:
        print(f"Error decoding JSON for URL: {url}")
        print(f"Response content: {page_content}")
        incidents = []

    incidents_table.extend(incidents)

    # Si no hay incidentes, asume que no hay más resultados y detiene el proceso
    if len(incidents) == 0:
        print(f"No incidents were returned. URL: {url}")
        print(f"End of incidents is assumed and process will be stopped.")
        break

# Guardar el resultado intermedio en un archivo JSON temporal
print("Saving intermediate result in a temporal JSON file")
with open('incidents_temp.json', 'w') as f:
    json.dump(incidents_table, f, indent=4)

https://cissm.liquifiedapps.com/api/?summary=1&start_row=0&end_row=100&filter={}&sort=[]
Failed to retrieve data from https://cissm.liquifiedapps.com/api/?summary=1&start_row=0&end_row=100&filter={}&sort=[]. HTTP Status Code: 503
https://cissm.liquifiedapps.com/api/?summary=1&start_row=100&end_row=200&filter={}&sort=[]
Failed to retrieve data from https://cissm.liquifiedapps.com/api/?summary=1&start_row=100&end_row=200&filter={}&sort=[]. HTTP Status Code: 503
https://cissm.liquifiedapps.com/api/?summary=1&start_row=200&end_row=300&filter={}&sort=[]
Failed to retrieve data from https://cissm.liquifiedapps.com/api/?summary=1&start_row=200&end_row=300&filter={}&sort=[]. HTTP Status Code: 503
https://cissm.liquifiedapps.com/api/?summary=1&start_row=300&end_row=400&filter={}&sort=[]
Failed to retrieve data from https://cissm.liquifiedapps.com/api/?summary=1&start_row=300&end_row=400&filter={}&sort=[]. HTTP Status Code: 503
https://cissm.liquifiedapps.com/api/?summary=1&start_row=400&end_row